In [ ]:
!pip install transformers
!pip install nltk

In [ ]:
# Загружаем датасет
#from datasets import load_dataset
#dataset = load_dataset("banking77")

# Загружаем модель
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

# Sentence separator
import nltk.data
nltk.download('punkt')
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [ ]:
from google.colab import drive
import pandas as pd 

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/My Drive/nlp_aug/5. Data/only_bad_aug_data.csv'
df = pd.read_csv(path)

In [ ]:
aug = df.values.tolist()

In [ ]:
len(aug)

4419

In [ ]:
# Пример

article = "How long should it take for my new card to arrive in them mail? What should I do if I never receive it?"
inputs = tokenizer(article, return_tensors="pt")

translated_tokens = model.generate(
    **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["kaz_Cyrl"], max_length=512
)
tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

['Жаңа картасыма хат келіп түсетінге қанша уақыт кетеді?']

In [ ]:
# Константы

KAZAKH = "kaz_Cyrl"
RUSSIAN = "rus_Cyrl"

def translate_sentence(text, language):
    article = text
    inputs = tokenizer(article, return_tensors="pt")

    translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[language], max_length=256)

    return tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

def translate_text(text, language):
    arr = sentence_tokenizer.tokenize(text)
    res = ""
    for a in arr:
        res += translate_sentence(a, language) + " "
    return res

In [ ]:
translate_text("How long should it take for my new card to arrive in them mail? What should I do if I never receive it?", KAZAKH)

'Жаңа картасы қанша уақытқа дейін поштамен келіп түседі? Егер оны ешқашан қабылдамасам, не істесем болады? '

In [ ]:
def demonstration(data):
  print('original: ', data)
  print('kazakh: ', translate_text(data, KAZAKH))

In [ ]:
for i in range(0, 1):
    print(i)
    a = dataset['test']['text'][i].replace(".", ".;")
    a = a.replace("!", "!;")
    if (a[len(a)-1] != ';'):
        a += ';'
    demonstration(a)
    print()

0
original:  How do I locate my card?;
kazakh:  Мен картамды қалай табамын? Ауыртпақ; 



In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
test = dataset['test']

In [ ]:
aug[:5]

[[0,
  0,
  'Is there a specific time frame within which I need to activate the newly received card?'],
 [1,
  0,
  'Could you tell me how long it takes for a new credit/debit card to be activated?'],
 [2,
  0,
  'My debit/credit cards have been lost, what should I do from here on-wards? How will they be reactivated?'],
 [3,
  0,
  "What happens if there is an error in activating a bank's debit or credit cards?"],
 [4,
  0,
  'Do I need to have a certain amount of money in my account before activating the card?']]

In [ ]:
translated = []

In [ ]:
for i in range(len(aug)):
    id = i
    label = aug[i][1]
    text = aug[i][2]
    result = translate_text(text, KAZAKH)
    res = {'id': id,'class':label,'text':text,'kazakh':result}
    translated.append(res)
    print(res)

{'id': 0, 'class': 0, 'text': 'Is there a specific time frame within which I need to activate the newly received card?', 'kazakh': 'Жаңадан алған картаны қосуға нақты уақыт бар ма? '}
{'id': 1, 'class': 0, 'text': 'Could you tell me how long it takes for a new credit/debit card to be activated?', 'kazakh': 'Жаңа кредиттік/дебет картасының белсендірілуіне қанша уақыт кетеді? '}
{'id': 2, 'class': 0, 'text': 'My debit/credit cards have been lost, what should I do from here on-wards? How will they be reactivated?', 'kazakh': 'Дебет/кредит карталарым жоғалып кетті, бұдан кейін не істеуім керек? Олар қалай қайтадан белсендіріледі? '}


KeyboardInterrupt: ignored

In [ ]:
import pandas as pd 
res_df = pd.DataFrame(translated)

In [ ]:
res_df

,id,class,text,kazakh
0,0,11,How do I locate my card?,Мен картамды қалай табамын?
1,1,11,"I still have not received my new card, I order...","Мен әлі күнге дейін жаңа картамды алмадым, бір..."
2,2,11,I ordered a card but it has not arrived. Help ...,"Мен карточка тапсырдым, бірақ ол келген жоқ. К..."
3,3,11,Is there a way to know when my card will arrive?,Картамның қашан келетінін білудің бір жолы бар...
4,4,11,My card has not arrived yet.,Менің карточкам әлі келген жоқ.
...,...,...,...,...
3075,3075,24,"If i'm not in the UK, can I still get a card?","Егер мен Ұлыбританияда болмаса, картаны аламын..."
3076,3076,24,How many countries do you support?,Сіз қанша елге қолдау көрсеттіңіз?
3077,3077,24,What countries do you do business in?,Сіз қандай елдерде бизнес жасайсыз?
3078,3078,24,What are the countries you operate in.,Сіз қандай елдерде жұмыс істейсіз?


In [ ]:
res_df.to_csv("translated_testcsv")